In [1]:
%cd ..

/home/wu/repo/gesture-wgan


In [2]:
import numpy as np
import pickle
from datasets.takekuchi.lowpass_filter import lowpass_filter
from takekuchi_dataset_tool.rot_to_pos import rot2pos

In [3]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity


def calculate_kde_batch(inputs, targets):
    inputs = np.concatenate(inputs, axis=0)
    targets = np.concatenate(targets, axis=0)
    mean, std, se = calculate_kde(inputs, targets)
    return mean, std, se


def calculate_kde(inputs, targets, verbose=0):
    params = {'bandwidth': np.logspace(-1, 1, 20)}
    grid = GridSearchCV(KernelDensity(), params, cv=3)
    grid.fit(inputs)
    kde = grid.best_estimator_
    scores = kde.score_samples(targets)
    mean = np.mean(scores)
    std = np.std(scores)
    se = std / np.sqrt(len(inputs))
    if verbose:
        print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))
        print(f"mean: {mean}, std: {std}, se: {se}")
    return mean, std, se

def compute_derivative(data):
    """(T, D)"""
    return data[1:] - data[:-1]

def compute_velocity(motion):
    '''(T, D)'''
    T = motion.shape[0]
    motion = motion.reshape(T, -1, 3)
    velocity = np.ndarray(shape=(0, motion.shape[1]))
    for t in range(1, T):
        vel = np.sqrt(np.sum((motion[t] - motion[t-1]) ** 2, axis=-1))
        velocity = np.append(velocity, vel[np.newaxis, :], axis=0)
    return velocity

In [5]:
with open("./data/takekuchi/processed/prosody/Y_dev.p", 'rb') as f:
    rots = pickle.load(f)

# rots = list(map(lowpass_filter, rots))

In [7]:
print("Evaluating rotation")
kde_rot, _, _ = calculate_kde_batch(rots, rots)

# Rotation velocity level
print("Evaluating rotation velocity")
rot_vel_outputs = list(map(compute_velocity, rots))
rot_vel_targets = list(map(compute_velocity, rots))
kde_rot_vel, _, _ = calculate_kde_batch(rot_vel_outputs, rot_vel_targets)

# Rotation acceleration level
print("Evaluating rotation acceleration")
rot_acc_outputs = list(map(compute_derivative, rot_vel_outputs))
rot_acc_targets = list(map(compute_derivative, rot_vel_targets))
kde_rot_acc, _, _ = calculate_kde_batch(rot_acc_outputs, rot_acc_targets)

# Position level
print("Evaluating position")
# pos_outputs = list(map(rot2pos, rots))
# pos_targets = list(map(rot2pos, rots))
# kde_pos, _, _ = calculate_kde_batch(pos_outputs, pos_targets)

# # Position velocity level
# print("Evaluating position velocity")
# pos_vel_outputs = list(map(compute_velocity, pos_outputs))
# pos_vel_targets = list(map(compute_velocity, pos_targets))
# kde_pos_vel, _, _ = calculate_kde_batch(pos_vel_outputs, pos_vel_targets)

# # Position acceleration level
# print("Evaluating position acceleration")
# pos_acc_outputs = list(map(compute_derivative, pos_vel_outputs))
# pos_acc_targets = list(map(compute_derivative, pos_vel_targets))
# kde_pos_acc, _, _ = calculate_kde_batch(pos_acc_outputs, pos_acc_targets)

Evaluating rotation
Evaluating rotation velocity
Evaluating rotation acceleration
Evaluating position


In [8]:
print("rot", kde_rot)
print("rot_vel", kde_rot_vel)
print("rot_acc", kde_rot_acc)
# print("pos", kde_pos)
# print("pos_vel", kde_pos_vel)
# print("pos_acc", kde_pos_acc)

rot -106.39945247107352
rot_vel -16.535067108637577
rot_acc -16.747412230005207
